<a href="https://colab.research.google.com/github/ByungjunKim/EnergyTransitionKorea/blob/main/%5BColab%5DCatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost
This code contains the CatBoost modeling and plotting visualization process used in the paper.

In [ ]:
!pip install -q catboost ipywidgets shap
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
# from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import shap
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200 # Increase to high DPI resolution.
# plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.sans-serif'] = ['sans-serif', 'DejaVu Sans', 'sans']
plt.rcParams['axes.unicode_minus'] = False
# plt.rcParams['text.usetex'] = True
shap.initjs()

In [ ]:
# Clone
!git clone https://github.com/ByungjunKim/EnergyTransitionKorea.git

In [ ]:
# Load the comment data from news articles related to energy transition.
df = pd.read_csv('./EnergyTransitionKorea/data/reply_df_catboost.csv')
df.head()

In [ ]:
df.columns

In [ ]:
# dropna
df = df.dropna().reset_index(drop=True)

In [ ]:
# Categorize news article titles.
df.loc[df['title_p/n']==-1.0,'title_p/n'] = 'negative'
df.loc[df['title_p/n']==0.0,'title_p/n'] = 'neutral'
df.loc[df['title_p/n']==1.0,'title_p/n'] = 'positive'
df['title_p/n'] = df['title_p/n'].astype('category')

In [ ]:
# Categorize news article policy.
df.loc[df['e_policy_p/n']==-1.0,'e_policy_p/n'] = 'negative'
df.loc[df['e_policy_p/n']==0.0,'e_policy_p/n'] = 'neutral'
df.loc[df['e_policy_p/n']==1.0,'e_policy_p/n'] = 'positive'
df['e_policy_p/n'] = df['e_policy_p/n'].astype('category')

In [ ]:
# Categorize user type (heavy or regular) as a nominal variable.
df['user_type'] = df['user_type'].astype('category')

### Model 1: Dependent variable "Condemning"

In [ ]:
# Creating data features.
X_1 = df[['user_type', 'title_p/n', 'e_policy_p/n',
         'politics', 'economy','society', 'culture', 'international', 'sports', 'IT_science', \
        'Condemning_past','Praising_past', 'Suffering_past', 'Self-Conscious_past',
        'tokens_len_past', 'ttr_past']]
y_1 = df['Condemning'].tolist()

In [ ]:
# Split into training and test sets
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.2, random_state=2023)

# Specify indices for categorical variables (0,1,2)
cat_features = [0,1,2]

In [ ]:
# Convert to CatBoost Pool format
train_pool_1 = Pool(X_train_1, y_train_1, cat_features=cat_features)
test_pool_1 = Pool(X_test_1, y_test_1, cat_features=cat_features)

In [ ]:
# Model initialization
model_1 = CatBoostRegressor(iterations=1000, learning_rate=0.1, verbose=200)

In [ ]:
# Train
model_1.fit(train_pool_1, eval_set=test_pool_1)

In [ ]:
# save model
# model_1.save_model('./catboost_model/condemning.model')

In [ ]:
# Predict
y_pred_1 = model_1.predict(X_test_1)

# R-squared
print("R-squared: {:.3f}".format(r2_score(y_test_1, y_pred_1)))

# Adjusted R2
n = X_test_1.shape[0]
k = X_test_1.shape[1]
adjusted_r2 = 1 - (1 - r2_score(y_test_1, y_pred_1)) * (n - 1) / (n - k - 1)
print("Adj R-squared: {:.3f}".format(adjusted_r2))

# MAPE
print("MAPE: {:.3f}".format(mean_absolute_percentage_error(y_test_1, y_pred_1)))

# Normalized MAE
# Calculate MAE
mae = np.mean(np.abs(y_test_1 - y_pred_1))
# Normalize MAE by the range of the dependent variable (Max - Min)
normalized_mae_range = mae / (np.max(y_test_1) - np.min(y_test_1))

# Normalize MAE by the standard deviation of the dependent variable
normalized_mae_std = mae / np.std(y_test_1)
print("Normalized MAE: {:.3f}".format(normalized_mae_range))
print("Normalized_std MAE: {:.3f}".format(normalized_mae_std))

# Calculate Coefficient of Variation of the RMSE (CV-RMSE)
print("CV-RMSE: {:.3f}".format(mean_squared_error(y_test_1, y_pred_1, squared=False) / np.mean(y_test_1)))

In [ ]:
# Get and plot SHAP values
explainer_1 = shap.Explainer(model_1)
shap_values_1 = explainer_1(X_1)
shap.summary_plot(shap_values_1, X_1,show=False)

# Setting the title with custom alignment
title = plt.title("a)", fontsize=15)
title.set_position([-0.5,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')
plt.show()

In [ ]:
# https://github.com/conorosully/medium-articles/blob/master/src/interpretable%20ml/SHAP/SHAP_catboost.ipynb

#Create for placeholder SHAP values
shap_values_cate_1 = explainer_1(X_1)

#Get shaply values and feature values for odor
odor_values = np.array(shap_values_1[:,'title_p/n'].values)
odor_data = X_1['title_p/n']

#Create new SHAP values array

#Split odor SHAP values by unique odor categories
unique_odor = list(X_1['title_p/n'].unique())
new_shap_values = [np.array(pd.Series(odor_values)[odor_data==odor]) for odor in unique_odor]

#Each sublist needs to be the same length
max_len = max([len(v) for v in new_shap_values])
new_shap_values = [np.append(vs,[np.nan]*(max_len - len(vs))) for vs in new_shap_values]
new_shap_values = np.array(new_shap_values)

#transpost matrix so categories are columns and SHAP values are rows
new_shap_values = new_shap_values.transpose()

#replace shap values
shap_values_cate_1.values = np.array(new_shap_values)

#replace data with placeholder array
shap_values_cate_1.data = np.array([[0]*len(unique_odor)]*max_len)

#replace base data with placeholder array
shap_values_cate_1.base = np.array([0]*max_len)

#replace feature names with category labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}
# labels = ["{} ({})".format(odor_labels[u],u) for u in unique_odor]
shap_values_cate_1.feature_names = unique_odor

#Use besswarm as before
shap.plots.beeswarm(shap_values_cate_1, color_bar=False,show=False, color='#808080')

# Setting the title with custom alignment
title = plt.title("b)", fontsize=15)
title.set_position([-0.18,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')

plt.show()

In [ ]:
#get shaply values and data
# odor_values = shap_values_1[:,'title_p/n'].values
# odor_data = X_1['title_p/n']
# unique_odor = set(X['title_p/n'])

#split odor shap values based on odor category
odor_categories = sorted(list(set(odor_data)))

odor_groups = []
for o in odor_categories:
    relevant_values = odor_values[odor_data == o]
    odor_groups.append(relevant_values)

# #replace categories with labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}

# labels = [odor_labels[u] for u in unique_odor]

#plot boxplot
plt.figure(figsize=(8, 5))

plt.boxplot(odor_groups,labels=odor_categories)

plt.ylabel('SHAP values (Condemning)',size=15)
plt.xlabel('title_p/n',size=15)

# Setting the title with custom alignment
title = plt.title("c)", fontsize=15)
title.set_position([-0.13,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')
plt.show()

### Model 2: Dependent variable "token_len"

In [ ]:
# Creating data features.
X_2 = df[['user_type', 'title_p/n', 'e_policy_p/n',
        'politics', 'economy','society', 'culture', 'international', 'sports', 'IT_science', \
        # 'Condemning',
        # 'Praising', 'Suffering','Self-Conscious',\
        'Condemning_past','Praising_past', 'Suffering_past', 'Self-Conscious_past',
        'tokens_len_past', 'ttr_past']]
y_2 = df['tokens_len']

In [ ]:
# Splitting data into training and testing sets
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=2023)

# Specifying indices of categorical features
cat_features = [0,1,2]

In [ ]:
# Convert to CatBoost Pool format
train_pool_2 = Pool(X_train_2, y_train_2, cat_features=cat_features)
test_pool_2 = Pool(X_test_2, y_test_2, cat_features=cat_features)

In [ ]:
# Model initialization
model_2 = CatBoostRegressor(iterations=1000, learning_rate=0.1, verbose=200)

In [ ]:
# Train
model_2.fit(train_pool_2, eval_set=test_pool_2)

In [ ]:
# model save
# model_2.save_model('./catboost_model/token_len.model')

In [ ]:
# Predict
y_pred_2 = model_2.predict(X_test_2)

# R-squared
print("R-squared: {:.3f}".format(r2_score(y_test_2, y_pred_2)))

# Adjusted R2
n = X_test_2.shape[0]
k = X_test_2.shape[1]
adjusted_r2 = 1 - (1 - r2_score(y_test_2, y_pred_2)) * (n - 1) / (n - k - 1)
print("Adj R-squared: {:.3f}".format(adjusted_r2))

# MAPE
print("MAPE: {:.3f}".format(mean_absolute_percentage_error(y_test_2, y_pred_2)))

# Normalized MAE
# Calculate MAE
mae = np.mean(np.abs(y_test_2 - y_pred_2))
# Normalize MAE by the range of the dependent variable (Max - Min)
normalized_mae_range = mae / (np.max(y_test_2) - np.min(y_test_2))

# Normalize MAE by the standard deviation of the dependent variable
normalized_mae_std = mae / np.std(y_test_2)
print("Normalized MAE: {:.3f}".format(normalized_mae_range))
print("Normalized_std MAE: {:.3f}".format(normalized_mae_std))

# Calculate Coefficient of Variation of the RMSE (CV-RMSE)
print("CV-RMSE: {:.3f}".format(mean_squared_error(y_test_2, y_pred_2, squared=False) / np.mean(y_test_2)))

In [ ]:
# Get and plot SHAP values
explainer_2 = shap.Explainer(model_2)
shap_values_2 = explainer_2(X_2)
shap.summary_plot(shap_values_2, X_2,show=False)

# Setting the title with custom alignment
title = plt.title("a)", fontsize=15)
title.set_position([-0.5,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')
plt.show()

In [ ]:
#Create for placeholder SHAP values
shap_values_cate_2 = explainer_2(X_2)

#Get shaply values and feature values for odor
odor_values = np.array(shap_values_2[:,'e_policy_p/n'].values)
odor_data = X_2['e_policy_p/n']

#Create new SHAP values array

#Split odor SHAP values by unique odor categories
unique_odor = list(X_2['e_policy_p/n'].unique())
new_shap_values = [np.array(pd.Series(odor_values)[odor_data==odor]) for odor in unique_odor]

#Each sublist needs to be the same length
max_len = max([len(v) for v in new_shap_values])
new_shap_values = [np.append(vs,[np.nan]*(max_len - len(vs))) for vs in new_shap_values]
new_shap_values = np.array(new_shap_values)

#transpost matrix so categories are columns and SHAP values are rows
new_shap_values = new_shap_values.transpose()

#replace shap values
shap_values_cate_2.values = np.array(new_shap_values)

#replace data with placeholder array
shap_values_cate_2.data = np.array([[0]*len(unique_odor)]*max_len)

#replace base data with placeholder array
shap_values_cate_2.base = np.array([0]*max_len)

#replace feature names with category labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}
# labels = ["{} ({})".format(odor_labels[u],u) for u in unique_odor]
shap_values_cate_2.feature_names = unique_odor

#Use besswarm as before
shap.plots.beeswarm(shap_values_cate_2, color_bar=False,show=False,color='#808080')

# Setting the title with custom alignment
title = plt.title("b)", fontsize=15)
title.set_position([-0.18,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')

plt.show()

In [ ]:
#get shaply values and data
# odor_values = shap_values[:,'e_policy_p/n'].values
# odor_data = X['e_policy_p/n']
# unique_odor = set(X['e_policy_p/n'])

#split odor shap values based on odor category
odor_categories = sorted(list(set(odor_data)))

odor_groups = []
for o in odor_categories:
    relevant_values = odor_values[odor_data == o]
    odor_groups.append(relevant_values)

# #replace categories with labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}

# labels = [odor_labels[u] for u in unique_odor]

#plot boxplot
plt.figure(figsize=(8, 5))

plt.boxplot(odor_groups,labels=odor_categories)

plt.ylabel('SHAP values (Token Len)',size=15)
plt.xlabel('e_policy_p/n',size=15)

# Setting the title with custom alignment
title = plt.title("c)", fontsize=15)
title.set_position([-0.13,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')

plt.show()

### Model 3: Dependent variable "ttr"

In [ ]:
# Creating data features.
X_3 = df[['user_type', 'title_p/n', 'e_policy_p/n', 'politics', 'economy',\
       'society', 'culture', 'international', 'sports', 'IT_science', \
        # 'Praising', 'Suffering','Self-Conscious',\
        'Condemning_past','Praising_past', 'Suffering_past', 'Self-Conscious_past',
        'tokens_len_past', 'ttr_past']]
y_3 = df['ttr']

In [ ]:
# Splitting data into training and testing sets
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.2, random_state=2023)

# Specifying indices of categorical features
cat_features = [0,1,2]

In [ ]:
# Convert to CatBoost Pool format.
train_pool_3 = Pool(X_train_3, y_train_3, cat_features=cat_features)
test_pool_3 = Pool(X_test_3, y_test_3, cat_features=cat_features)

In [ ]:
# Model initialization
model_3 = CatBoostRegressor(iterations=1000, learning_rate=0.1, verbose=200)

In [ ]:
# Train
model_3.fit(train_pool_3, eval_set=test_pool_3)

In [ ]:
# model save
# model_3.save_model('./catboost_model/token_ttr.model')

In [ ]:
# Predict
y_pred_3 = model_3.predict(X_test_3)

# R-squared
print("R-squared: {:.3f}".format(r2_score(y_test_3, y_pred_3)))

# Adjusted R2
n = X_test_3.shape[0]
k = X_test_3.shape[1]
adjusted_r2 = 1 - (1 - r2_score(y_test_3, y_pred_3)) * (n - 1) / (n - k - 1)
print("Adj R-squared: {:.3f}".format(adjusted_r2))

# MAPE
print("MAPE: {:.3f}".format(mean_absolute_percentage_error(y_test_3, y_pred_3)))

# Normalized MAE
# Calculate MAE
mae = np.mean(np.abs(y_test_3 - y_pred_3))
# Normalize MAE by the range of the dependent variable (Max - Min)
normalized_mae_range = mae / (np.max(y_test_3) - np.min(y_test_3))

# Normalize MAE by the standard deviation of the dependent variable
normalized_mae_std = mae / np.std(y_test_3)
print("Normalized MAE: {:.3f}".format(normalized_mae_range))
print("Normalized_std MAE: {:.3f}".format(normalized_mae_std))

# Calculate Coefficient of Variation of the RMSE (CV-RMSE)
print("CV-RMSE: {:.3f}".format(mean_squared_error(y_test_3, y_pred_3, squared=False) / np.mean(y_test_3)))

In [ ]:
# Get and plot SHAP values
explainer_3 = shap.Explainer(model_3)
shap_values_3 = explainer_3(X_3)
shap.summary_plot(shap_values_3, X_3, show=False)

# Setting the title with custom alignment
title = plt.title("a)", fontsize=15)
title.set_position([-0.5,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')
plt.show()

In [ ]:
#Create for placeholder SHAP values
shap_values_cate_3 = explainer_3(X_3)

#Get shaply values and feature values for odor
odor_values = np.array(shap_values_3[:,'e_policy_p/n'].values)
odor_data = X_3['e_policy_p/n']

#Create new SHAP values array

#Split odor SHAP values by unique odor categories
unique_odor = list(X_3['e_policy_p/n'].unique())
new_shap_values = [np.array(pd.Series(odor_values)[odor_data==odor]) for odor in unique_odor]

#Each sublist needs to be the same length
max_len = max([len(v) for v in new_shap_values])
new_shap_values = [np.append(vs,[np.nan]*(max_len - len(vs))) for vs in new_shap_values]
new_shap_values = np.array(new_shap_values)

#transpost matrix so categories are columns and SHAP values are rows
new_shap_values = new_shap_values.transpose()

#replace shap values
shap_values_cate_3.values = np.array(new_shap_values)

#replace data with placeholder array
shap_values_cate_3.data = np.array([[0]*len(unique_odor)]*max_len)

#replace base data with placeholder array
shap_values_cate_3.base = np.array([0]*max_len)

#replace feature names with category labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}
# labels = ["{} ({})".format(odor_labels[u],u) for u in unique_odor]
shap_values_cate_3.feature_names = unique_odor

#Use besswarm as before
shap.plots.beeswarm(shap_values_cate_3, color_bar=False,show=False,color='#808080')

# Setting the title with custom alignment
title = plt.title("b)", fontsize=15)
title.set_position([-0.18,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')

plt.show()

In [ ]:
#get shaply values and data
# odor_values = shap_values[:,'e_policy_p/n'].values
# odor_data = X['e_policy_p/n']
# unique_odor = set(X['e_policy_p/n'])

#split odor shap values based on odor category
odor_categories = sorted(list(set(odor_data)))

odor_groups = []
for o in odor_categories:
    relevant_values = odor_values[odor_data == o]
    odor_groups.append(relevant_values)

# #replace categories with labels
# odor_labels = {'a':'almond',
#                'l':'anise',
#                'c':'creosote',
#                'y':'fishy',
#                'f':'foul',
#                'm':'musty',
#                'n':'none',
#                'p':'pungent',
#                's':'spicy'}

# labels = [odor_labels[u] for u in unique_odor]

#plot boxplot
plt.figure(figsize=(8, 5))

plt.boxplot(odor_groups,labels=odor_categories)

plt.ylabel('SHAP values (TTR)',size=15)
plt.xlabel('e_policy_p/n',size=15)

# Setting the title with custom alignment
title = plt.title("c)", fontsize=15)
title.set_position([-0.13,1]) # You can adjust the [0, 1.02] values as needed
title.set_ha('left')

plt.show()